In [2]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import re
from nltk.corpus import stopwords
import nltk
import string
import matplotlib.pyplot as plt
import random
%matplotlib inline
import os
import json

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn import decomposition, ensemble
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
#from nltk.corpus import stopwords
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix

import joblib

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import os,boto3,sys,glob,json

### Part 1  Loading the  data 

#### Read in csv file

In [3]:
#testDF=pd.read_csv('run1000charts_0521.csv')
testDF= pd.read_csv('/home/jovyan/work/Analytics_Data_training/bfAI_allinst_7502charts_061019.csv') # 7502 charts
# the csv is generated in D drive D:/chartai_qa  with extract1000charts_jsontxt.py, then uploaded to here

In [4]:
len(testDF)

418518

In [5]:
testDF.columns

Index(['Unnamed: 0', 'chart_id', 'code', 'text', 'start_offset', 'end_offset',
       'text75'],
      dtype='object')

In [6]:
testDF=testDF.drop(['Unnamed: 0'],axis=1)

In [7]:
testDF.head()

,chart_id,code,text,start_offset,end_offset,text75
0,IBXRMA2018_MULT_1218010190010001_HMK_188263480...,E785,Hyperlipidemia,2526,2540,b12 deficiency vitamin deficiency intertrigo b...
1,IBXRMA2018_MULT_1218010190010001_HMK_188263480...,I10,Hypertension,2543,2555,vitamin deficiency intertrigo bmi 350359adult ...
2,IBXRMA2018_MULT_1218010190010001_HMK_188263480...,G20,Parkinson's Disease,3258,3277,parkinsons fu history present illness 82 yo wh...
3,IBXRMA2018_MULT_1218010190010001_HMK_188263480...,E46,deficiency,3972,3982,elchair bound minimal transfer bradykinesis ac...
4,IBXRMA2018_MULT_1218010190010001_HMK_188263480...,E550,vitamin d deficiency,2468,2488,er quadrant abdominal pelvic swelling mass lum...


In [8]:
# just check if there's null text, if yes, remove them 
nulltext=testDF[~pd.notnull(testDF['text'])] 
len(nulltext)

0

In [9]:
nulltext=testDF[~pd.notnull(testDF['text75'])] 
len(nulltext)

208

In [10]:
nulltext   # there are some text not correctly extracted

,chart_id,code,text,start_offset,end_offset,text75
93,IBXRMA2018_MULT_1216944220010001_HMK_195147709...,E744,PC,25,27,NaN
2812,IBXRMA2018_MULT_1217060910010001_HMK_097164250...,E744,PC,25,27,NaN
5677,IBXRMA2018_MULT_1216129800010001_HMK_239481844...,G129,SMA,4,7,NaN
7858,IBXRMA2018_MULT_1218015600010001_HMK_3GM6V72QA...,N08,Kidney,48,54,NaN
8082,IBXRMA2018_MULT_1216901800010001_HMK_201224196...,G129,SMA,4,7,NaN
10147,IBXRMA2018_MULT_1217591910010001_HMK_168240489...,E780,cholesterol,60,71,NaN
14436,IBXRMA2018_MULT_1216874020010001_HMK_143364782...,E744,PC,25,27,NaN
18295,IBXRMA2018_MULT_1217464990010001_HMK_175243784...,N08,Kidney,46,52,NaN
21856,IBXRMA2018_MULT_1216920010010001_HMK_166407506...,N08,Kidney,46,52,NaN
21988,IBXRMA2018_MULT_1217403970010001_HMK_166200321...,E744,PC,25,27,NaN


In [11]:
testDF=testDF[pd.notnull(testDF['text75'])] 
len(testDF)

418310

In [12]:
testDF['chart_id'].nunique()   # only 980 charts not 1044 charts

7500

In [13]:
testDF=testDF.reset_index(drop=True)   # very very import, to reset the index, otherwise the below running the model will not be right, because I use the index as reference for position and matching them
testDF.tail()

,chart_id,code,text,start_offset,end_offset,text75
418305,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,I482,atrial,367638,367644,left atrium mildly dilated left atrium right a...
418306,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,I482,atrial,367668,367674,ed left atrium right atrium normal right atria...
418307,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,E041,thyroid nodule,371470,371484,nable raise left arm 13 cm hypodense left lowe...
418308,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,E041,thyroid nodule,372201,372215,icant lytic blastic bone lesions impression 13...
418309,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,I509,congestive heart failure,374108,374132,robably gross acute pathology examination siza...


### Part 2 Loading all the models or locate the directory

#### Part 2.a. Load the 28 pickle files and run them on the testDF with selected chart_ids

In [14]:
#files=glob.glob("../../picklefolder_ngrams/refreshed_models_update0531/*.pickle") # refreshed updated models
files=glob.glob("../../picklefolder_ngrams/refreshed_models/*19.pickle")  # refreshed models


In [15]:
files

['../../picklefolder_ngrams/refreshed_models/Phrase_stroke_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_emphysema_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_hypertension_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_dementia_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cholesterol_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cad_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_aneurysm_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_COPD_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Depression_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Diabetes_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Heart_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Kidney_LRApril0219.pickle',
 '../../picklefolder_ngrams/refr

In [16]:
files2=glob.glob("../../picklefolder_ngrams/refreshed_models_update0531/*.pickle") # refreshed updated models
#files2=files2[:-1]   # run 28 model
files2=files2[28]   # run consolidated only

In [17]:
files.append(files2)

In [18]:
files

['../../picklefolder_ngrams/refreshed_models/Phrase_stroke_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_emphysema_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_hypertension_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_dementia_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cholesterol_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cad_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_aneurysm_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_COPD_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Depression_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Diabetes_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Heart_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Kidney_LRApril0219.pickle',
 '../../picklefolder_ngrams/refr

In [19]:
files[27].split('/')[4].split('_')[1]

'preventative'

In [20]:
files[0].split('/')[4]

'Phrase_stroke_LRApril0219.pickle'

### Part 3. Apply models to test data

In [21]:
#encoder = preprocessing.LabelEncoder()
#test_y = encoder.fit_transform(testDF['flag'])  


In [22]:
code_list={}
code_list["Model1"]=['I10','E119']
code_list["Model2"]=['I2510','E785','K219','F329','I639']
code_list["Model3"]=['I739','N183','Z992','B20','I213','R569','I43','J410','I714','R579','E550','I209','J45998','I480','B182','K210','K739']
code_list["Heart"]=['I509','I482','I4891','I82401']
code_list["Depression"]=['F411','F410','F40240','F458','F409','F4000','F4001','F445','F444','F40243','F446','F4010','F442','F408','F449','F451']
code_list["Kidney"]=['N189','N181','N182','E1122','I130','E0822','E1022']
code_list["Diabetes"]=['Z794','E109','E119','E139','E089','E099']
code_list["Cancer"]=['C801','C50919','C189']
code_list["COPD"]=['J449','J45909']
code_list["Cholesterol"]=['E780','E785','E782','E789','E7800','E784','E781','E7801','E7881','E882','E783','E786','E7889','E756','E755','E7130']

In [23]:
allpredict=pd.DataFrame()   # create an empty dataframe, to save the prediction result for each model
allprob=pd.DataFrame()
print(allprob)

Empty DataFrame
Columns: []
Index: []


In [24]:
# for one model evaluation
'''
indexdf=pd.DataFrame() 
model_pipeline = joblib.load(files)
  #  f = open("allmodeltestscore_0506.txt", "a+")
    # deletion model

testDF1=testDF
indexdf[0]= [1 for i in range(len(testDF))]
        
        
        #print(files[i])
    # how to put the prediction on the right index?
test=testDF1['text75']
print("model loaded:", files )
print("length of data for the model:", len(test))
    
   
proba = model_pipeline.predict_proba(test)[:,1]
prediction = np.where(proba > 0.3, 1, 0) # use a lower threshold
testDF1['key']=testDF1.index
testDF1['pred']=prediction
testDF1['prob']= proba

indexdf['key']=indexdf.index
merged=pd.merge(indexdf,testDF1,on='key',how='outer')  # maintain the position of the perdiction, match to the right index
allpredict[0]=merged['pred']    # save the prediction to corresponding dataframe
allprob[0]=merged['prob']
'''

'\nindexdf=pd.DataFrame() \nmodel_pipeline = joblib.load(files)\n  #  f = open("allmodeltestscore_0506.txt", "a+")\n    # deletion model\n\ntestDF1=testDF\nindexdf[0]= [1 for i in range(len(testDF))]\n        \n        \n        #print(files[i])\n    # how to put the prediction on the right index?\ntest=testDF1[\'text75\']\nprint("model loaded:", files )\nprint("length of data for the model:", len(test))\n    \n   \nproba = model_pipeline.predict_proba(test)[:,1]\nprediction = np.where(proba > 0.3, 1, 0) # use a lower threshold\ntestDF1[\'key\']=testDF1.index\ntestDF1[\'pred\']=prediction\ntestDF1[\'prob\']= proba\n\nindexdf[\'key\']=indexdf.index\nmerged=pd.merge(indexdf,testDF1,on=\'key\',how=\'outer\')  # maintain the position of the perdiction, match to the right index\nallpredict[0]=merged[\'pred\']    # save the prediction to corresponding dataframe\nallprob[0]=merged[\'prob\']\n'

In [25]:
#f = open("allmodeltestscore_0506.txt", "w+")  # save all the score to this file
# for multiple model evaluation

   # create a dataframe, otherwise can use testDF for merging to obtain the prediction matched to index
# because for some model testing, we only select partial data, like phrase-based model, code-based model
for i in range(len(files)):
    indexdf=pd.DataFrame() 
    model_pipeline = joblib.load(files[i])
  #  f = open("allmodeltestscore_0506.txt", "a+")
    # deletion model
    if(files[i].split('/')[4].startswith('D')):
        try:
            testDF1=testDF
            indexdf[i]= [1 for i in range(len(testDF))]
        except:
            print('no data for this model')
        
    elif(files[i].split('/')[4].startswith('P')):
    # if it's phrase-based model, use the following way to select data
        
        phrase=files[i].split('/')[4].split('_')[1]
        try:
            testDF1 = testDF[testDF['text'].str.contains(phrase,flags=re.IGNORECASE)]
            temp=testDF1.index.tolist()
            indexdf[i]=[1 if i in (temp) else 0 for i in range(len(testDF))]
        except:
            print('no data for this model')

    elif(files[i].split('/')[4].startswith('C')):
    # if it's code based model, use the following way to select model
        codereason=files[i].split('/')[4].split('_')[1]
        try:
            testDF1 = testDF[testDF['code'].isin(code_list[codereason])]
            temp=testDF1.index.tolist()
            indexdf[i]=[1 if i in (temp) else 0 for i in range(len(testDF))]
        except:
            print('no data for this model')
    else:
        print('not a valid/good model name probably...')

    #print(files[i])
    # how to put the prediction on the right index?
    test=testDF1['text75']
    print("length of data for the model:", len(test))
    try:
        
        proba = model_pipeline.predict_proba(test)[:,1]
        prediction = np.where(proba > 0.4, 1, 0) # use a lower threshold
        #prediction = model_pipeline.predict(test)
        
        
        testDF1['key']=testDF1.index
        testDF1['pred']=prediction
        testDF1['prob']= proba

        indexdf['key']=indexdf.index
        merged=pd.merge(indexdf,testDF1,on='key',how='outer')  # maintain the position of the perdiction, match to the right index
        allpredict[i]=merged['pred']    # save the prediction to corresponding dataframe
        allprob[i]=merged['prob']   # save the probability
    except:
         print('something is wrong')


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your ow

length of data for the model: 1623


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

length of data for the model: 964


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your ow

...

length of data for the model: 418310


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your ow

length of data for the model: 418310


/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:251: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.2 when using version 0.20.2. This might lead to breaking code or invalid results. Use at your ow

length of data for the model: 418310
length of data for the model: 418310


####  Part 2.b. Now get the prediction and probability dataframes and merge with testDF which has all the chart_id etc info

In [26]:
len(allpredict)

418310

In [27]:
allpredict.isnull().sum()  # a lot of nulls because not all data are run for each model

0     416687
1     417346
2     379038
3     415581
4     400572
5     411683
6     415678
7     407652
8     411347
9     383801
10    400817
11    411935
12    331013
13    408093
14    372160
15    357800
16    387081
17    414331
18    403546
19    415178
20    409148
21    391235
22         0
23         0
24         0
25         0
26         0
27         0
28         0
dtype: int64

In [28]:
allpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,0,0,0,0


In [29]:
allprob.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.001588,0.009432,0.014130,0.034596,0.001052,0.002447,0.067671
1,NaN,NaN,0.094929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.003035,0.007828,0.018645,0.027765,0.001430,0.002414,0.074822
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.115857,0.033108,0.007716,0.067311,0.001899,0.025832,0.129453
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.003465,0.032156,0.071882,0.167899,0.009440,0.013970,0.203773
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.003773,0.049663,0.043583,0.169887,0.003179,0.019628,0.135165


In [30]:
len(allprob)

418310

In [31]:
len(testDF)

418310

In [32]:
# the data frame for predictions from each model
#merge to testDF to keep the Nan, which means not omputed for that instance
#testDF2=pd.concat([testDF,allpredict],axis=1)
#len(testDF2)
testDF['index']=testDF.index
allpredict['index']=allpredict.index
df_pred = pd.merge(testDF,allpredict, how='left', on = 'index')
len(df_pred)

418310

In [33]:
df_pred.tail()

,chart_id,code,text,start_offset,end_offset,text75,key,pred,prob,index,...,19,20,21,22,23,24,25,26,27,28
418305,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,I482,atrial,367638,367644,left atrium mildly dilated left atrium right a...,418305,1,0.932658,418305,...,NaN,NaN,NaN,0,0,1,1,0,0,1
418306,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,I482,atrial,367668,367674,ed left atrium right atrium normal right atria...,418306,1,0.936481,418306,...,NaN,NaN,NaN,0,0,1,1,0,0,1
418307,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,E041,thyroid nodule,371470,371484,nable raise left arm 13 cm hypodense left lowe...,418307,1,0.768912,418307,...,NaN,NaN,NaN,0,0,1,1,0,0,1
418308,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,E041,thyroid nodule,372201,372215,icant lytic blastic bone lesions impression 13...,418308,1,0.678415,418308,...,NaN,NaN,NaN,0,1,1,0,0,0,1
418309,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,I509,congestive heart failure,374108,374132,robably gross acute pathology examination siza...,418309,1,0.817968,418309,...,NaN,NaN,NaN,0,1,1,0,0,0,1


In [34]:
# now the data frame with probabilities
testDF['index']=testDF.index
allprob['index']=allprob.index
df_prob = pd.merge(testDF,allprob, how='left', on = 'index')
print(len(df_prob))

418310


In [35]:
df_prob.tail()

,chart_id,code,text,start_offset,end_offset,text75,key,pred,prob,index,...,19,20,21,22,23,24,25,26,27,28
418305,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,I482,atrial,367638,367644,left atrium mildly dilated left atrium right a...,418305,1,0.932658,418305,...,NaN,NaN,NaN,0.008212,0.279688,0.846352,0.804261,0.027458,0.051732,0.932658
418306,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,I482,atrial,367668,367674,ed left atrium right atrium normal right atria...,418306,1,0.936481,418306,...,NaN,NaN,NaN,0.005806,0.298900,0.850264,0.837606,0.022916,0.043327,0.936481
418307,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,E041,thyroid nodule,371470,371484,nable raise left arm 13 cm hypodense left lowe...,418307,1,0.768912,418307,...,NaN,NaN,NaN,0.008633,0.121493,0.715362,0.547000,0.056095,0.020721,0.768912
418308,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,E041,thyroid nodule,372201,372215,icant lytic blastic bone lesions impression 13...,418308,1,0.678415,418308,...,NaN,NaN,NaN,0.007722,0.668864,0.720203,0.318181,0.008026,0.026901,0.678415
418309,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,I509,congestive heart failure,374108,374132,robably gross acute pathology examination siza...,418309,1,0.817968,418309,...,NaN,NaN,NaN,0.008685,0.435160,0.438354,0.325368,0.015158,0.021450,0.817968


#### Part 2.c. Now combine the prediction from 28 models to make one prediction,it's actually an or operation, but here using sum()

In [36]:
allprob.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,index
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.001588,0.009432,0.014130,0.034596,0.001052,0.002447,0.067671,0
1,NaN,NaN,0.094929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.003035,0.007828,0.018645,0.027765,0.001430,0.002414,0.074822,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.115857,0.033108,0.007716,0.067311,0.001899,0.025832,0.129453,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.003465,0.032156,0.071882,0.167899,0.009440,0.013970,0.203773,3
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.003773,0.049663,0.043583,0.169887,0.003179,0.019628,0.135165,4


In [37]:
allpredictNan=allpredict   # keep the original NaN if that row is not computed for that model
allpredict=allpredict.fillna(0) # fill the NaNs by zero

In [38]:
allpredictNan.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,index
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,0
1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,3
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,4


In [39]:
allpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,index
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,4


In [40]:
allpredict.drop(['index'],inplace=True, axis=1)
allpredict.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [41]:
allpredict['Sum'] = allpredict.sum(axis = 1, skipna = True) 
df_pred['all_model_pred']=[1 if allpredict['Sum'].iloc[i]>=1 else 0 for i in range(len(allpredict))] 

In [42]:
# check how many 1s are predicted 
allpredict1=df_pred['all_model_pred']
print(sum(allpredict1))
print(len(allpredict1))  # 228562 are predicted as deleted out of 418310

225651
418310


In [43]:
# drop the key and predict prob temp columns
df_pred.drop(['key','pred','prob','index'],inplace=True, axis=1)
df_prob.drop(['key','pred','prob','index'],inplace=True, axis=1)

In [44]:
df_pred.tail()
#all_model_pred is check any model output is 1 for that instance

,chart_id,code,text,start_offset,end_offset,text75,0,1,2,3,...,20,21,22,23,24,25,26,27,28,all_model_pred
418305,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,I482,atrial,367638,367644,left atrium mildly dilated left atrium right a...,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,1,1,0,0,1,1
418306,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,I482,atrial,367668,367674,ed left atrium right atrium normal right atria...,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,1,1,0,0,1,1
418307,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,E041,thyroid nodule,371470,371484,nable raise left arm 13 cm hypodense left lowe...,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,1,1,0,0,1,1
418308,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,E041,thyroid nodule,372201,372215,icant lytic blastic bone lesions impression 13...,NaN,NaN,NaN,NaN,...,NaN,NaN,0,1,1,0,0,0,1,1
418309,IBXRMA2018_MULT_1217052600010001_HMK_1UH0MM2YA...,I509,congestive heart failure,374108,374132,robably gross acute pathology examination siza...,NaN,NaN,NaN,NaN,...,NaN,NaN,0,1,1,0,0,0,1,1


In [45]:
df_pred.iloc[76761]   # just check one instance

chart_id          IBXRMA2018_MULT_1217111440010001_HMK_159289263...
code                                                           I509
text                                                  Heart Failure
start_offset                                                  81201
end_offset                                                    81214
text75            healthcare provider discharge instructions pat...
0                                                               NaN
1                                                               NaN
2                                                               NaN
3                                                               NaN
4                                                               NaN
5                                                               NaN
6                                                               NaN
7                                                               NaN
8                                               

In [46]:
df_pred.columns

Index([      'chart_id',           'code',           'text',   'start_offset',
           'end_offset',         'text75',                0,                1,
                      2,                3,                4,                5,
                      6,                7,                8,                9,
                     10,               11,               12,               13,
                     14,               15,               16,               17,
                     18,               19,               20,               21,
                     22,               23,               24,               25,
                     26,               27,               28, 'all_model_pred'],
      dtype='object')

In [47]:
df_prob.columns

Index([    'chart_id',         'code',         'text', 'start_offset',
         'end_offset',       'text75',              0,              1,
                    2,              3,              4,              5,
                    6,              7,              8,              9,
                   10,             11,             12,             13,
                   14,             15,             16,             17,
                   18,             19,             20,             21,
                   22,             23,             24,             25,
                   26,             27,             28],
      dtype='object')

In [48]:
files

['../../picklefolder_ngrams/refreshed_models/Phrase_stroke_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_emphysema_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_hypertension_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_dementia_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cholesterol_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_cad_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Phrase_aneurysm_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_COPD_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Depression_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Diabetes_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Heart_LRApril0219.pickle',
 '../../picklefolder_ngrams/refreshed_models/Code_Kidney_LRApril0219.pickle',
 '../../picklefolder_ngrams/refr

In [49]:
# rename the columns names, has to be consistent with the above order
# since there's no 0, or 1, 17, these models didn't run because no data for them

col_dict = {0: 'Phrase_stroke', 1: 'Phrase_emphysema',2:'Phrase_hypertension',3:'Phrase_dementia',4:'Phrase_cholesterol',
           5:'Phrase_cad',6:'Phrase_aneurysm',7:'Code_COPD',8:'Code_Depression',9:'Code_Diabetes',10:'Code_Heart',
           11:'Code_Kidney',12:'Code_Model1',13:'Code_Cancer',14:'Code_Cholesterol',15:'Code_Model2',
           16:'Code_Model3',17:'Phrase_asthma',18:'Phrase_cancer',19:'Phrase_ckd',20:'Phrase_depression',21:'Phrase_diabetes',
            22:'DeletionReason_family_history',23:'DeletionReason_negation',24:'DeletionReason_not_doctors_note',
           25:'DeletionReason_not_relevant',26:'DeletionReason_phoneORaddressORother_number',27:'DeletionReason_preventative_screening',
           }   ## key→old name, value→new name

df_pred.columns = [col_dict.get(x, x) for x in df_pred.columns]
df_prob.columns = [col_dict.get(x, x) for x in df_prob.columns]


In [50]:
df_pred.columns

Index([                                   'chart_id',
                                              'code',
                                              'text',
                                      'start_offset',
                                        'end_offset',
                                            'text75',
                                     'Phrase_stroke',
                                  'Phrase_emphysema',
                               'Phrase_hypertension',
                                   'Phrase_dementia',
                                'Phrase_cholesterol',
                                        'Phrase_cad',
                                   'Phrase_aneurysm',
                                         'Code_COPD',
                                   'Code_Depression',
                                     'Code_Diabetes',
                                        'Code_Heart',
                                       'Code_Kidney',
                            

In [51]:
df_prob.columns

Index([                                   'chart_id',
                                              'code',
                                              'text',
                                      'start_offset',
                                        'end_offset',
                                            'text75',
                                     'Phrase_stroke',
                                  'Phrase_emphysema',
                               'Phrase_hypertension',
                                   'Phrase_dementia',
                                'Phrase_cholesterol',
                                        'Phrase_cad',
                                   'Phrase_aneurysm',
                                         'Code_COPD',
                                   'Code_Depression',
                                     'Code_Diabetes',
                                        'Code_Heart',
                                       'Code_Kidney',
                            

In [52]:
#df_pred.rename(columns={'start': 'start_offset', 'end': 'end_offset'}, inplace=True)
#df_pred[['chart_id','start_offset','end_offset','DeletionReason_family_history']].tail()

In [53]:
#df_prob.rename(columns={'start': 'start_offset', 'end': 'end_offset'}, inplace=True)
#df_prob[['chart_id','start_offset','end_offset','DeletionReason_family_history']].tail()  #ok make sense

In [54]:
#df_pred.to_csv('run1000charts_pred_0521.csv')   #  with prediction for each model and all model combined
#df_prob.to_csv('probdataforcompwithAM_0506.csv')   #  with probability for each model
# finally corrected a lot of errors like index not reset issue

In [55]:
print(len(set(df_pred['chart_id'].tolist())))
print(len(set(df_prob['chart_id'].tolist())))   # 813 unique chart ids

7500
7500


###  Part 3. Load in coder feedback and  evaluate the accuracy

#### Select common chart_ids from both 1000 model run and 6000 coder annotation

In [56]:
# model computation
df_pred1=df_pred[['chart_id', 'code', 'text', 'start_offset', 'end_offset', 'text75','all_model_pred']]
print(len(df_pred1))
df_pred1 = df_pred1.drop_duplicates()
print(len(df_pred1))

418310
418310


In [57]:
# read in all charts with coder feedback
#df_codeall = pd.read_csv('/home/jovyan/work/Analytics_Data_training/conditions_with_chartids_29042019.csv')
df_codeall = pd.read_csv('/home/jovyan/work/Analytics_Data_training/conditions_output_withchart_id_100619.csv')
print(len(df_codeall))

188444


In [58]:
print("unique chart is in 1000 charts run model output:",df_pred1['chart_id'].nunique())
print("unique chart is in 6000 charts with coder feedback:",df_codeall['chart_id'].nunique())

unique chart is in 1000 charts run model output: 7500
unique chart is in 6000 charts with coder feedback: 7654


In [59]:
# now find overlap chart ids
overlapped_charts = pd.merge(df_codeall[['chart_id']].drop_duplicates(),df_pred1[['chart_id']].drop_duplicates(), how = 'inner')
overlapped_charts = list(overlapped_charts['chart_id'].unique())
print(len(overlapped_charts))  

7500


In [60]:
# select the model output
fd_prod = df_pred1[['chart_id','code','start_offset','end_offset','text75','all_model_pred']][df_pred1['chart_id'].isin(overlapped_charts)].drop_duplicates()
len(fd_prod)

418310

In [61]:
# select those in the code output
data = df_codeall[df_codeall['chart_id'].isin(overlapped_charts)]
print(len(data))
print(len(data[data['label'] == 'added']))
print(len(data[data['label'] == 'deleted']))
print(len(data[data['label'] == 'agreed']))

187681
109958
25331
52392


In [62]:
# The pipeline TN can be computed as, the following is not true negative yet
TNh= len(fd_prod)-(len(data)-len(data[data['label'] == 'added']))
print("TNh:",TNh)
TNh= len(fd_prod)-(len(data)-len(data[data['label'] == 'deleted']))
print("TNh:",TNh)

TNh: 340587
TNh: 255960


#### The accurary number for the pipeline models

In [68]:
# Use fuzzy logic to compute the deleted, added, agreed
data = data[data['start'] != 0]
data = data[data['end'] != 0]
#data = data[data['created_at'] == date ] ##Change date to filter for datewise data
#label_counts = data.groupby(['label']).size().reset_index(name='counts')
instmatches = data[data['label'] == 'agreed']
instfuzzymatches = data[data['label'] == 'deleted']
instfuzzymatches = instfuzzymatches[instfuzzymatches['deleted_reason'].isin(['Incorrect Specification - Non-Risk Adjusted','repeated_instance','incorrect_year_of_service']) ]
instmatches = pd.concat([instmatches, instfuzzymatches])
instmatches = instmatches[['chart_id','submission_id','code_id','start','end']].drop_duplicates()

instdeleted = data[data['label'] == 'deleted']
instdeleted = instdeleted[instdeleted['deleted_reason'] != 'other']
instdeleted = pd.merge(instdeleted, instmatches,how = 'outer', on = ['chart_id','submission_id','code_id','start','end'], indicator = True)
instdeleted = instdeleted[instdeleted['_merge'] == 'left_only']
instdeleted = instdeleted[['chart_id','submission_id','code_id','start','end','deleted_reason']].drop_duplicates()
deleted_reason = instdeleted.groupby(['deleted_reason']).size().reset_index(name = 'counts')
instdeleted = instdeleted[['chart_id','submission_id','code_id','start','end']].drop_duplicates()


instadded = data[data['label'] == 'added']
instadded = pd.merge(instadded, instmatches,how = 'outer', on = ['chart_id','submission_id','code_id','start','end'], indicator = True)
instadded = instadded[instadded['_merge'] == 'left_only']
instadded = instadded[['chart_id','submission_id','code_id','start','end']].drop_duplicates()
instadded = pd.merge(instadded, instdeleted,how = 'outer', on = ['chart_id','submission_id','code_id','start','end'], indicator = True)
instadded = instadded[instadded['_merge'] == 'left_only']
instadded = instadded[['chart_id','submission_id','code_id','start','end']].drop_duplicates()

added = len(instadded)
deleted = len(instdeleted)
agreed = len(instmatches)
print(added)
print(deleted)
print(agreed)

98442
4930
55232


In [69]:
instmatches.columns

Index(['chart_id', 'submission_id', 'code_id', 'start', 'end'], dtype='object')

In [70]:
# my added small part
instmatchessub=instmatches[['chart_id', 'code_id', 'submission_id','start','end']]
instmatchessub.columns = ['chart_id', 'code_id','submission_id', 'start_offset', 'end_offset']

In [71]:
fd_prod['all_model_pred'].value_counts()

1    225651
0    192659
Name: all_model_pred, dtype: int64

In [72]:
# should use fd_prod[fd_prod['all_model_pred']==1]  if it want to find instadded_modeldeleted
instadded.columns = ['chart_id', 'submission_id', 'code_id', 'start_offset', 'end_offset']
prod_additions_models = pd.merge(instadded, fd_prod,how = 'inner', on = ['chart_id'], indicator = True)
prod_additions_models['abs_diff_start'] = abs(prod_additions_models['start_offset_x'] - prod_additions_models['start_offset_y'])
prod_additions_models['abs_diff_end'] = abs(prod_additions_models['end_offset_x'] - prod_additions_models['end_offset_y'])
prod_additions_models1 = prod_additions_models[ (
                    (prod_additions_models['code_id'] == prod_additions_models['code']) 
                    & (prod_additions_models['abs_diff_start'] <= 75))
          | (prod_additions_models['start_offset_x'] == prod_additions_models['start_offset_y'])
          | (prod_additions_models['end_offset_x'] == prod_additions_models['end_offset_y'])
          |((prod_additions_models['code_id'] == prod_additions_models['code']) 
                    & (prod_additions_models['abs_diff_end'] <= 75))
          ]
instadded_modeldeleted = prod_additions_models1[['chart_id','submission_id','code','start_offset_y','end_offset_y']].drop_duplicates()

In [73]:
len(instadded_modeldeleted)  # should it be called instadded_modelpassed?

93889

In [74]:
# my correction here
instadded.columns = ['chart_id', 'submission_id', 'code_id', 'start_offset', 'end_offset']
fd_prod1=fd_prod[fd_prod['all_model_pred']==1] # if it want to find instadded_modeldeleted

prod_additions_models = pd.merge(instadded, fd_prod1,how = 'inner', on = ['chart_id'], indicator = True)
prod_additions_models['abs_diff_start'] = abs(prod_additions_models['start_offset_x'] - prod_additions_models['start_offset_y'])
prod_additions_models['abs_diff_end'] = abs(prod_additions_models['end_offset_x'] - prod_additions_models['end_offset_y'])
prod_additions_models1 = prod_additions_models[ (
                    (prod_additions_models['code_id'] == prod_additions_models['code']) 
                    & (prod_additions_models['abs_diff_start'] <= 75))
          | (prod_additions_models['start_offset_x'] == prod_additions_models['start_offset_y'])
          | (prod_additions_models['end_offset_x'] == prod_additions_models['end_offset_y'])
          |((prod_additions_models['code_id'] == prod_additions_models['code']) 
                    & (prod_additions_models['abs_diff_end'] <= 75))
          ]
instadded_modeldeleted2 = prod_additions_models1[['chart_id','submission_id','code','start_offset_y','end_offset_y']].drop_duplicates()
print(len(instadded_modeldeleted2))
# it differs dramatically from istadded_modeldeleted, because istadded_modeldeleted is taking the whole instances before AI
# overlap with the added. If that's the purpose, shouldn't be called the misleading istadded_modeldeleted

18730


In [75]:
# initial numbers considering all the additions data for recall

total = len(instmatches) + len(instdeleted) + len(instadded)
added = len(instadded)
deleted = len(instdeleted)
agreed = len(instmatches)

TP = agreed
FN = added
FP = deleted
TN= len(fd_prod)-(len(data)-len(data[data['label'] == 'added']))


precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)

print('-------------------')
print('Agreements = ', agreed)
print('Deletions = ', deleted)
print('Additions = ', added)

TP: 55232
TN: 340587
FN: 98442
FP: 4930
precision: 0.9180545859512649
recall: 0.3594101799914104
accuracy: 0.7929209460907749
-------------------
Agreements =  55232
Deletions =  4930
Additions =  98442


In [68]:
# initial numbers considering model deleted additions data for recall
'''

total = len(instmatches) + len(instdeleted) + len(instadded_modeldeleted)
added = len(instadded_modeldeleted)  # shouldn't use here, because this model is not the orginal model, 
                                    
deleted = len(instdeleted)
agreed = len(instmatches)

TP = agreed
FN = added
FP = deleted
TN1= len(fd_prod)-(len(data)-len(data[data['label'] == 'added']))

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN1+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)

print('-------------------')
print('Agreements = ', agreed)
print('Deletions = ', deleted)
print('Additions = ', added)
'''

TP: 55232
TN: 340587
FN: 93889
FP: 4930
precision: 0.9180545859512649
recall: 0.3703837822975973
accuracy: 0.8002195545024846
-------------------
Agreements =  55232
Deletions =  4930
Additions =  93889


####  Refreshed  Model accuracy numbers

In [76]:
print(len(df_pred1))   # model output with all the instances

418310


In [77]:
print(len(fd_prod))  # already selected  the 7500 chart  from previous steps

418310


In [78]:
fd_prod['chart_id'].nunique()

7500

In [79]:
fd_prod.dtypes

chart_id          object
code              object
start_offset       int64
end_offset         int64
text75            object
all_model_pred     int64
dtype: object

In [80]:
TN=len(fd_prod[fd_prod['all_model_pred']==1])  # predicted deleted instances, but not true negative yet
print("TN:",TN)

TN: 225651


In [81]:
# select those 7500 charts and only passed the filtering not deleted, 0 is keep here
dev_df_sub_1 = fd_prod[fd_prod['all_model_pred']==0]  # select the predicted as pass instances
len(dev_df_sub_1[['chart_id','start_offset']].drop_duplicates())  # 192659 instances pass through

192659

In [82]:
dev_df_sub_1.rename(columns={'start_offset':'start','end_offset':'end'},inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [83]:
dev_df_sub_1.rename(columns={'code':'code_id'},inplace=True)

In [95]:
# selected the model predicted as deleted instances
# 1 is delete here
dev_df_sub_2 = fd_prod[fd_prod['all_model_pred']==1]  # select the predicted as model delete instances
print(len(dev_df_sub_2[['chart_id','start_offset']].drop_duplicates()))  # 225651 instances didn't pass through
dev_df_sub_2.rename(columns={'start_offset':'start','end_offset':'end'},inplace=True)
dev_df_sub_2.rename(columns={'code':'code_id'},inplace=True)

225651


In [96]:
modeldelete_coderagreed=pd.merge(dev_df_sub_2[['chart_id','start']], instmatches,how = 'inner', on = ['chart_id','start'], indicator = True)
print(len(modeldelete_coderagreed))

3187


In [97]:
modeldelete_coderadded=pd.merge(dev_df_sub_2[['chart_id','start']], instadded,how = 'inner', on = ['chart_id','start'], indicator = True)
print(len(modeldelete_coderadded))

14297


In [84]:
new_matches = pd.merge(dev_df_sub_1[['chart_id','start']], instmatches,how = 'inner', on = ['chart_id','start'], indicator = True)
new_deleted = pd.merge(dev_df_sub_1[['chart_id','start']], instdeleted,how = 'inner', on = ['chart_id','start'], indicator = True)

In [85]:
instadded.columns = ['chart_id', 'submission_id', 'code_id', 'start', 'end']
new_added = pd.merge(dev_df_sub_1, instadded,how = 'inner', on = ['chart_id'], indicator = True)
new_added['abs_diff_start'] = abs(new_added['start_x'] - new_added['start_y'])
new_added['abs_diff_end'] = abs(new_added['end_x'] - new_added['end_y'])

In [86]:
new_added1 = new_added[ (
                    (new_added['code_id_x'] == new_added['code_id_y']) 
                    & (new_added['abs_diff_start'] <= 5))
          | (new_added['start_x'] == new_added['start_y'])
          ]
print(len(new_added1))

55113


In [98]:
# or use fuzzy match
newmodeldeleted_added = pd.merge(dev_df_sub_2, instadded,how = 'inner', on = ['chart_id'], indicator = True)
newmodeldeleted_added['abs_diff_start'] = abs(new_added['start_x'] - new_added['start_y'])
newmodeldeleted_added['abs_diff_end'] = abs(new_added['end_x'] - new_added['end_y'])
newmodeldeleted_added1 = newmodeldeleted_added[ (
                    (newmodeldeleted_added['code_id_x'] == newmodeldeleted_added['code_id_y']) 
                    & (newmodeldeleted_added['abs_diff_start'] <= 75))
          | (newmodeldeleted_added['start_x'] == newmodeldeleted_added['start_y'])
          | (newmodeldeleted_added['end_x'] == newmodeldeleted_added['end_y'])
          |((newmodeldeleted_added['code_id_x'] == newmodeldeleted_added['code_id_y']) 
                    & (newmodeldeleted_added['abs_diff_end'] <= 75))
          ]
print(len(newmodeldeleted_added1))

27737


In [100]:
# use fuzzy match there are more instances for model-deleted but coder added, from 14297 to 27737
modeldelete_coderadded2=newmodeldeleted_added1
print(len(modeldelete_coderadded2))

27737


In [102]:
#1. if the start offset matches exactly
#2. If the end offset matches exactly
#3. If the overlap of start offset is between 75 characters and the code id matches
#4. If the overlap of end offset is between 75 characters and the code id matches
new_added2 = new_added[ (
                    (new_added['code_id_x'] == new_added['code_id_y']) 
                    & (new_added['abs_diff_start'] <= 75))
          | (new_added['start_x'] == new_added['start_y'])
          | (new_added['end_x'] == new_added['end_y'])
          |((new_added['code_id_x'] == new_added['code_id_y']) 
                    & (new_added['abs_diff_end'] <= 75))
          ]
print(len(new_added2))

90795


In [103]:
instadded_modeldeleted.columns

Index(['chart_id', 'submission_id', 'code_id', 'start', 'end'], dtype='object')

In [104]:
#just validating the additions with a different method of calculation
instadded_modeldeleted.columns = ['chart_id', 'submission_id', 'code_id', 'start', 'end']
x = pd.merge(dev_df_sub_1[['chart_id','start']].drop_duplicates()
             , instadded_modeldeleted[['chart_id','start']].drop_duplicates()
             ,how = 'inner', on = ['chart_id','start'], indicator = True)
len(x)

74464

In [105]:
instadded_modeldeleted2.columns

Index(['chart_id', 'submission_id', 'code_id', 'start', 'end'], dtype='object')

In [106]:
#just validating the additions with a different method of calculation
instadded_modeldeleted2.columns = ['chart_id', 'submission_id', 'code_id', 'start', 'end']
x2 = pd.merge(dev_df_sub_1[['chart_id','start']].drop_duplicates()
             , instadded_modeldeleted2[['chart_id','start']].drop_duplicates()
             ,how = 'inner', on = ['chart_id','start'], indicator = True)
len(x2)    # overlap model deleted, instance added, with the model agreed instances, should be zero? Added by my check

0

In [107]:
print(len(new_matches))
print(len(new_deleted))
print(len(new_added1[['chart_id','start_x']].drop_duplicates()))
print(len(new_added2[['chart_id','start_x']].drop_duplicates()))

52043
3692
54188
74464


In [109]:
#type 1 - older number of additions considering only start offset
refreshed_agreements1 = int(len(new_matches)) + int(len(new_added1[['chart_id','start_x']].drop_duplicates()))
refreshed_deletions1 = int(len(dev_df_sub_1[['chart_id','start']].drop_duplicates())) - (int(len(new_matches)) + int(len(new_added1[['chart_id','start_x']].drop_duplicates()))) 
refreshed_additions1 = added - int(len(new_added1[['chart_id','start_x']].drop_duplicates()))

#type 2 - new number of additions considering start offset and end offset
refreshed_agreements2 = int(len(new_matches)) + int(len(new_added2[['chart_id','start_x']].drop_duplicates()))
refreshed_deletions2 = int(len(dev_df_sub_1[['chart_id','start']].drop_duplicates())) - (int(len(new_matches)) + int(len(new_added2[['chart_id','start_x']].drop_duplicates()))) 
refreshed_additions2 = added - int(len(new_added2[['chart_id','start_x']].drop_duplicates()))

# have second thought on addition, should we use model predicted as deletion, but coder feedback is agreed or added
# as deletion? I'm adding it here as refreshed_additions3

refreshed_additions3= int(len(modeldelete_coderagreed))+ int(len(modeldelete_coderadded2))
refreshed_additions4= int(len(modeldelete_coderagreed))+ int(len(modeldelete_coderadded))

print(refreshed_agreements1)
print(refreshed_deletions1)
print(refreshed_additions1)
print("--------")
print(refreshed_agreements2)
print(refreshed_deletions2)
print(refreshed_additions2)
print(refreshed_additions3)  # this one defintely make the number worse, because it's the FN
print(refreshed_additions4)

106231
86428
44254
--------
126507
66152
23978
30924
17484


In [111]:
refreshed_total1 = refreshed_agreements1 + refreshed_deletions1 + refreshed_additions1
refreshed_total2 = refreshed_agreements2 + refreshed_deletions2 + refreshed_additions2
refreshed_total3 = refreshed_agreements2 + refreshed_deletions2 + refreshed_additions3
refreshed_total4 = refreshed_agreements2 + refreshed_deletions2 + refreshed_additions4
print(refreshed_total1)
print(refreshed_total2)
print(refreshed_total3)
print(refreshed_total4)

236913
216637
223583
210143


In [112]:
# my added part of calculating TN
fd_prodsub=fd_prod[fd_prod['all_model_pred']==1]
fd_prodsub2=fd_prod[fd_prod['all_model_pred']==0]
print(len(fd_prodsub))   # predicted as delete
print(len(fd_prodsub2))  # not deleted, presented to coder
fd_prodsub.columns=['chart_id', 'code_id', 'start_offset', 'end_offset', 'text75',
       'all_model_pred']   # model predicted as deleted
fd_prodsub2.columns=['chart_id', 'code_id', 'start_offset', 'end_offset', 'text75',
       'all_model_pred']   # model predicted as passing through or agree

225651
192659


In [113]:
instaddedsub=instadded[['chart_id', 'submission_id','code_id', 'start','end']]
instaddedsub.columns = ['chart_id', 'submission_id', 'code_id', 'start_offset', 'end_offset']

In [114]:
fd_prodsub.columns=['chart_id', 'code_id', 'start_offset', 'end_offset', 'text75',
       'all_model_pred']

In [115]:
instmatchedaddedsub = pd.concat([instaddedsub,instmatchessub])
#instmatchedaddedsub=instmatchedaddedsub[['chart_id', 'code_id', 'submission_id','start','end']]
#instmatchedaddedsub.columns = ['chart_id', 'code_id','submission_id', 'start_offset', 'end_offset']
#print(len(instmatchedaddedsub))  # 25273
prod_agree_models = pd.merge(instmatchedaddedsub, fd_prodsub,how = 'inner', on = ['chart_id'], indicator = True)
len(prod_agree_models)

prod_agree_models['abs_diff_start'] = abs(prod_agree_models['start_offset_x'] - prod_agree_models['start_offset_y'])
prod_agree_models['abs_diff_end'] = abs(prod_agree_models['end_offset_x'] - prod_agree_models['end_offset_y'])

prod_agree_models1 = prod_agree_models[ (
                    (prod_agree_models['code_id_x'] == prod_agree_models['code_id_y']) 
                    & (prod_agree_models['abs_diff_start'] <= 5))
          | (prod_agree_models['start_offset_x'] == prod_agree_models['start_offset_y'])
          | (prod_agree_models['end_offset_x'] == prod_agree_models['end_offset_y'])
          |((prod_agree_models['code_id_x'] == prod_agree_models['code_id_y']) 
                    & (prod_agree_models['abs_diff_end'] <= 5))
          ]

instaddagreed_modeldeleted = prod_agree_models1[['chart_id','submission_id','code_id_x','code_id_y','start_offset_x','end_offset_x']].drop_duplicates()
print(len(instaddagreed_modeldeleted))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


20220

In [116]:
#type 1 calculation

print('Instance level New Models:')
print('Agreements = ', refreshed_agreements1*100/refreshed_total1,'%')
print('Deletions = ', refreshed_deletions1*100/refreshed_total1,'%')
print('Additions = ', refreshed_additions1*100/refreshed_total1,'%')
print('Precision = ', refreshed_agreements1*100/(refreshed_agreements1+refreshed_deletions1),'%')
print('Recall = ', refreshed_agreements1*100/(refreshed_agreements1+refreshed_additions1),'%')
print('-------------------')
print('Agreements = ', refreshed_agreements1)
print('Deletions = ', refreshed_deletions1)
print('Additions = ', refreshed_additions1)

TP = refreshed_agreements1
FP = refreshed_deletions1
FN = refreshed_additions1

TN=len(fd_prod[fd_prod['all_model_pred']==1])-len(instaddagreed_modeldeleted)  # deleted instances

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)


Instance level New Models:
Agreements =  44.839666881935564 %
Deletions =  36.480902272142096 %
Additions =  18.679430845922344 %
Precision =  55.1393913598638 %
Recall =  70.59241784895505 %
-------------------
Agreements =  106231
Deletions =  86428
Additions =  44254
TP: 106231
TN: 205431
FN: 44254
FP: 86428
precision: 0.551393913598638
recall: 0.7059241784895505
accuracy: 0.7045692944857396


In [117]:
#type 2 calculation

print('Instance level New Models:')
print('Agreements = ', refreshed_agreements2*100/refreshed_total2,'%')
print('Deletions = ', refreshed_deletions2*100/refreshed_total2,'%')
print('Additions = ', refreshed_additions2*100/refreshed_total2,'%')
print('Precision = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_deletions2),'%')
print('Recall = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_additions2),'%')
print('-------------------')
print('Agreements = ', refreshed_agreements2)
print('Deletions = ', refreshed_deletions2)
print('Additions = ', refreshed_additions2)



TP = refreshed_agreements2
FP = refreshed_deletions2
FN = refreshed_additions2

TN=len(fd_prod[fd_prod['all_model_pred']==1])-len(instaddagreed_modeldeleted)  # deleted instances

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)



Instance level New Models:
Agreements =  58.395841892197545 %
Deletions =  30.53587337343113 %
Additions =  11.068284734371321 %
Precision =  65.66368557918395 %
Recall =  84.06618599860451 %
-------------------
Agreements =  126507
Deletions =  66152
Additions =  23978
TP: 126507
TN: 205431
FN: 23978
FP: 66152
precision: 0.6566368557918395
recall: 0.8406618599860451
accuracy: 0.7864562108475411


In [119]:
#type 3 calculation

print('Instance level New Models:')
print('Agreements = ', refreshed_agreements2*100/refreshed_total3,'%')
print('Deletions = ', refreshed_deletions2*100/refreshed_total3,'%')
print('Additions = ', refreshed_additions3*100/refreshed_total3,'%')
print('Precision = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_deletions2),'%')
print('Recall = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_additions3),'%')
print('-------------------')
print('Agreements = ', refreshed_agreements2)
print('Deletions = ', refreshed_deletions2)
print('Additions = ', refreshed_additions3)



TP = refreshed_agreements2
FP = refreshed_deletions2
FN = refreshed_additions3

TN=len(fd_prod[fd_prod['all_model_pred']==1])-len(instaddagreed_modeldeleted)  # deleted instances

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)


Instance level New Models:
Agreements =  56.58167213070761 %
Deletions =  29.58722264215079 %
Additions =  13.831105227141599 %
Precision =  65.66368557918395 %
Recall =  80.35710882862969 %
-------------------
Agreements =  126507
Deletions =  66152
Additions =  30924
TP: 126507
TN: 205431
FN: 30924
FP: 66152
precision: 0.6566368557918395
recall: 0.8035710882862969
accuracy: 0.7737230020465533


In [120]:
#type 4 calculation

print('Instance level New Models:')
print('Agreements = ', refreshed_agreements2*100/refreshed_total4,'%')
print('Deletions = ', refreshed_deletions2*100/refreshed_total4,'%')
print('Additions = ', refreshed_additions3*100/refreshed_total4,'%')
print('Precision = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_deletions2),'%')
print('Recall = ', refreshed_agreements2*100/(refreshed_agreements2+refreshed_additions4),'%')
print('-------------------')
print('Agreements = ', refreshed_agreements2)
print('Deletions = ', refreshed_deletions2)
print('Additions = ', refreshed_additions4)



TP = refreshed_agreements2
FP = refreshed_deletions2
FN = refreshed_additions4

TN=len(fd_prod[fd_prod['all_model_pred']==1])-len(instaddagreed_modeldeleted)  # deleted instances

precision=TP/(FP+TP)
recall=TP/(FN+TP)
accuracy=(TP+TN)/(TP+TN+FN+FP)

print('TP:',TP)
print('TN:',TN)
print('FN:',FN)
print('FP:',FP)
print('precision:',precision)
print('recall:',recall)
print('accuracy:',accuracy)

Instance level New Models:
Agreements =  60.2004349419205 %
Deletions =  31.479516329356677 %
Additions =  14.715693599120598 %
Precision =  65.66368557918395 %
Recall =  87.85757443173532 %
-------------------
Agreements =  126507
Deletions =  66152
Additions =  17484
TP: 126507
TN: 205431
FN: 17484
FP: 66152
precision: 0.6566368557918395
recall: 0.8785757443173532
accuracy: 0.7987458310673912


In [91]:
# save a html copy
#!jupyter nbconvert --to html all28refreshedmodels_comptoAMresult_050819-deleteinstances.ipynb